In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers

import time
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow import keras
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
from qnn_utils import get_mnist_data, get_quantum_data, CNNModel

## Load Data

In [3]:
train_images, val_images, test_images, train_labels, val_labels, test_labels = get_mnist_data()

## Learn Models

In [4]:
statistics = {"n_layers":[], "n_wires":[], "time_processing":[], "Score":[]}

In [5]:
%%time
# train Classic NN
c_model = CNNModel()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/ClassicModel", histogram_freq=1)
# train
c_history = c_model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                        batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
# validate
results = c_model.evaluate(test_images, test_labels, batch_size=64)
# update statistics
statistics["n_layers"].append("Classic")
statistics["n_wires"].append("Classic")
statistics["time_processing"].append("Classic")
statistics["Score"].append(results[1])

8/8 [==============================] - 0s 11ms/step - loss: 0.0884 - accuracy: 0.9820
Wall time: 1min 13s


In [6]:
%%time
for n_layers in range(1, 4):
    for n_wires in range(1, 6):
        # get quantum data
        preprocess_time, q_train_images, q_val_images, q_test_images = get_quantum_data(train_images, val_images,
                                                                                        test_images, n_layers, n_wires)
        # train quantum NN
        q_model = CNNModel()
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/QNNLayers_{n_layers}Wires_{n_wires}",
                                                              histogram_freq=1)
        q_history = q_model.fit(q_train_images, train_labels, validation_data=(q_val_images, val_labels),
                                batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
        # validate
        results = q_model.evaluate(q_test_images, test_labels, batch_size=64)
        # update statistics
        statistics["n_layers"].append(n_layers)
        statistics["n_wires"].append(n_wires)
        statistics["time_processing"].append(preprocess_time)
        statistics["Score"].append(results[1])
        # pirnt params
        print(f"Layers: {n_layers}, Wires: {n_wires}, Score: {results[1]}")

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

8/8 [==============================] - 0s 10ms/step - loss: 0.0663 - accuracy: 0.9840
Layers: 1, Wires: 1, Score: 0.984000027179718


  0%|          | 0/4000 [00:00<?, ?it/s]

KeyboardInterrupt: 